In [1]:
import pandas as pd

In [62]:
# 加载数据
train = pd.read_csv('./train_10000.csv')
valid_auth = pd.read_csv('validate_1000.csv')

# 绘图代码，可随时调用，用于查看数据预处理效果

In [64]:
# 将训练集的所有数据绘制散点图，存储在train
import matplotlib.pyplot as plt
for feature in range(1,107):
    train_label_0 = train.loc[train['label'] == 0]
    train_label_1 = train.loc[train['label'] == 1]
    train_label_2 = train.loc[train['label'] == 2]
    train_label_3 = train.loc[train['label'] == 3]
    train_label_4 = train.loc[train['label'] == 4]
    train_label_5 = train.loc[train['label'] == 5]
    plt.figure(figsize=(16, 9))
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)
    plt.scatter(train_label_0['sample_id'],train_label_0.iloc[:,[feature]],s=0.2,c='blue', facecolor='grey')
    plt.scatter(train_label_1['sample_id'],train_label_1.iloc[:,[feature]],s=0.2,c='black', facecolor='grey')
    plt.scatter(train_label_2['sample_id'],train_label_2.iloc[:,[feature]],s=0.2,c='green', facecolor='grey')
    plt.scatter(train_label_3['sample_id'],train_label_3.iloc[:,[feature]],s=0.2,c='red', facecolor='grey')
    plt.scatter(train_label_4['sample_id'],train_label_4.iloc[:,[feature]],s=0.2,c='orange', facecolor='grey')
    plt.scatter(train_label_5['sample_id'],train_label_5.iloc[:,[feature]],s=0.2,c='purple', facecolor='grey')
    plt.savefig(f'imputed_train/{feature-1}.png')
    plt.close()

In [14]:
# 将验证集的所有数据绘制散点图，存储在validation文件夹
import matplotlib.pyplot as plt
for feature in range(1,107):
    valid_label_0 = valid_auth.loc[valid_auth['label'] == 0]
    valid_label_1 = valid_auth.loc[valid_auth['label'] == 1]
    valid_label_2 = valid_auth.loc[valid_auth['label'] == 2]
    valid_label_3 = valid_auth.loc[valid_auth['label'] == 3]
    valid_label_4 = valid_auth.loc[valid_auth['label'] == 4]
    valid_label_5 = valid_auth.loc[valid_auth['label'] == 5]
    plt.figure(figsize=(16, 9))
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)
    plt.scatter(valid_label_0['sample_id'],valid_label_0.iloc[:,[feature]],s=2,c='blue', facecolor='grey')
    plt.scatter(valid_label_1['sample_id'],valid_label_1.iloc[:,[feature]],s=2,c='black', facecolor='grey')
    plt.scatter(valid_label_2['sample_id'],valid_label_2.iloc[:,[feature]],s=2,c='green', facecolor='grey')
    plt.scatter(valid_label_3['sample_id'],valid_label_3.iloc[:,[feature]],s=2,c='red', facecolor='grey')
    plt.scatter(valid_label_4['sample_id'],valid_label_4.iloc[:,[feature]],s=2,c='orange', facecolor='grey')
    plt.scatter(valid_label_5['sample_id'],valid_label_5.iloc[:,[feature]],s=2,c='purple', facecolor='grey')
    # plt.scatter(valid_auth['sample_id'],valid_auth.iloc[:,[feature]],s=0.2,c='red')
    plt.savefig(f'validation/{feature-1}.png')
    plt.close()

# 使用四分位点判别法，对训练集的label==0和label==1的值进行降噪处理，然后采用KNN填充缺失值。

这是第一种四分位判别法，基于label==0和label==1来确定离群值的阈值，然后应用这些阈值

In [51]:
import pandas as pd
import numpy as np

# 计算整个数据集的四分位数
Q1 = train.iloc[:, 1:107].quantile(0.25)
Q3 = train.iloc[:, 1:107].quantile(0.75)
IQR = Q3 - Q1

# 需要处理的label
noisy_labels = [0, 1]

for label in noisy_labels:
    # 获取该label的数据
    df_label = train[train['label'] == label].copy()

    # 对每一列特征分别进行处理
    for col in df_label.columns[1:107]:
        # 将离群值设为NaN
        df_label.loc[(df_label[col] < Q1[col] - 1.5 * IQR[col]) | (df_label[col] > Q3[col] + 1.5 * IQR[col]), col] = np.nan

    # 更新原始数据
    train.loc[train['label'] == label] = df_label

# 你可以选择用KNN imputer或其他方法处理NaN值，或者直接删除含有NaN的行
# df.dropna(inplace=True)
train

,sample_id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature98,feature99,feature100,feature101,feature102,feature103,feature104,feature105,feature106,label
0,0,51.567250,288358.4,NaN,1.855900,201.460169,6.582261,-0.516321,5.636771e+11,2.222212e+05,...,31459.628135,254.582034,0.0,-2.014506e+08,159.299350,0.603211,NaN,-5.256075,180.977310,4
1,1,63.804874,288358.4,1.106802,1.050387,391.605375,13.323439,4.662871,NaN,-1.442474e+05,...,42830.526855,270.580779,0.0,-1.534970e+09,NaN,0.506220,0.552654,16.505952,314.783263,5
2,2,49.138527,288358.4,1.111649,0.767127,130.708067,6.485547,5.696815,5.474603e+11,-4.288403e+05,...,249963.241809,160.207067,0.0,7.998345e+08,112.632639,0.080100,0.235920,64.707581,183.304610,2
3,3,NaN,288358.4,1.109169,NaN,258.521076,0.716737,23.238461,-3.539891e+11,NaN,...,125478.297201,196.223295,0.0,-9.246920e+07,138.431470,2.548783,1.414810,-9.662399,212.302670,2
4,4,76.520831,288358.4,1.113410,5.795408,256.038997,-1.803483,14.040495,-1.071014e+11,6.499723e+05,...,186266.423019,179.083883,0.0,-6.796351e+08,259.858740,0.337643,0.228832,59.733069,135.541233,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,66.948837,288358.4,1.102805,NaN,260.805938,6.663322,-6.495112,-1.959797e+11,-1.169449e+06,...,131837.380427,207.050530,0.0,-5.375408e+08,NaN,1.598658,1.239977,-1.625234,NaN,0
9996,9996,NaN,288358.4,1.107632,0.679847,NaN,22.327405,2.251901,8.157356e+11,1.887989e+05,...,3416.075865,228.252191,0.0,4.302888e+08,NaN,2.348464,3.079268,11.018358,250.312228,0
9997,9997,42.208350,288358.4,1.110253,0.037611,180.875482,9.471924,6.725166,1.039534e+12,8.453963e+04,...,NaN,211.716051,0.0,-5.853507e+07,167.836714,3.840799,NaN,75.537477,241.702576,2
9998,9998,26.797447,288358.4,1.094471,0.505019,253.278224,19.199080,-4.451117,5.994078e+11,-3.025228e+05,...,101045.903091,225.324419,0.0,7.009934e+08,27.189454,0.538561,NaN,-10.481948,113.104089,0


这是第二种四分位判别法，基于整个数据集来确定离群值的阈值，然后在各自的label中（例如label==0和label==1）应用这些阈值。

In [63]:
import numpy as np

# 需要处理的label
noisy_labels = [0, 1]

for label in noisy_labels:
    # 获取该label的数据
    df_label = train[train['label'] == label].copy()

    # 对每一列特征分别进行处理
    for col in df_label.columns[1:107]:
        Q1 = df_label[col].quantile(0.25)
        Q3 = df_label[col].quantile(0.75)
        IQR = Q3 - Q1

        # 将离群值设为NaN
        df_label.loc[(df_label[col] < Q1 - 1.5 * IQR) | (df_label[col] > Q3 + 1.5 * IQR), col] = np.nan

    # 更新原始数据
    train.loc[train['label'] == label] = df_label
train

以上两种方法只能采用其中一种，但是我并不确定哪种比较好。建议最后两种都试试，看看谁在验证集跑的分高。

## KNN插值算法

In [60]:
from sklearn.impute import KNNImputer

# 获取所有的label
labels = train['label'].unique()

# 对于每一个label，分别进行插值
for label in labels:
    # 获取该label的数据
    df_label = train[train['label'] == label]

    # 取出feature列
    features = df_label.iloc[:,1:107]

    # 创建KNN imputer
    imputer = KNNImputer(n_neighbors=5)

    # 对feature进行插值
    features_imputed = imputer.fit_transform(features)

    # 更新原始数据
    train.loc[train['label'] == label, train.columns[1:107]] = features_imputed

# 最后，train就是插值后的数据
train

,sample_id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature98,feature99,feature100,feature101,feature102,feature103,feature104,feature105,feature106,label
0,0,51.567250,288358.4,1.106507,1.855900,201.460169,6.582261,-0.516321,5.636771e+11,2.222212e+05,...,31459.628135,254.582034,0.0,-2.014506e+08,159.299350,0.603211,0.387657,-5.256075,180.977310,4
1,1,63.804874,288358.4,1.106802,1.050387,391.605375,13.323439,4.662871,1.089855e+11,-1.442474e+05,...,42830.526855,270.580779,0.0,-1.534970e+09,163.755351,0.506220,0.552654,16.505952,314.783263,5
2,2,49.138527,288358.4,1.111649,0.767127,130.708067,6.485547,5.696815,5.474603e+11,-4.288403e+05,...,249963.241809,160.207067,0.0,7.998345e+08,112.632639,0.080100,0.235920,64.707581,183.304610,2
3,3,64.411529,288358.4,1.109169,2.580881,258.521076,0.716737,23.238461,-3.539891e+11,6.907655e+05,...,125478.297201,196.223295,0.0,-9.246920e+07,138.431470,2.548783,1.414810,-9.662399,212.302670,2
4,4,76.520831,288358.4,1.113410,5.795408,256.038997,-1.803483,14.040495,-1.071014e+11,6.499723e+05,...,186266.423019,179.083883,0.0,-6.796351e+08,259.858740,0.337643,0.228832,59.733069,135.541233,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,66.948837,288358.4,1.102805,-0.310484,260.805938,6.663322,-6.495112,-1.959797e+11,-1.169449e+06,...,131837.380427,207.050530,0.0,-5.375408e+08,148.080110,1.598658,1.239977,-1.625234,NaN,0
9996,9996,41.137296,288358.4,1.107632,0.679847,160.449641,22.327405,2.251901,8.157356e+11,1.887989e+05,...,3416.075865,228.252191,0.0,4.302888e+08,131.923443,2.348464,3.079268,11.018358,250.312228,0
9997,9997,42.208350,288358.4,1.110253,0.037611,180.875482,9.471924,6.725166,1.039534e+12,8.453963e+04,...,149650.953476,211.716051,0.0,-5.853507e+07,167.836714,3.840799,0.834373,75.537477,241.702576,2
9998,9998,26.797447,288358.4,1.094471,0.505019,253.278224,19.199080,-4.451117,5.994078e+11,-3.025228e+05,...,101045.903091,225.324419,0.0,7.009934e+08,27.189454,0.538561,2.199572,-10.481948,113.104089,0


# 对这些特征进行Anova方差分析，提取有效的50个特征